In [1]:
import torch
from qer.ai.hemorrhages import Hemorrhages2DBlend
import pandas as pd
from monai.data import Dataset
from torch.utils.data import DataLoader
import monai
import random
from typing import Any, Dict, List, Sequence, Union, Tuple, Optional

import numpy as np
import SimpleITK as sitk
import torch
import time
from loguru import logger
from monai.transforms import (
    DivisiblePadd,
    RandomizableTransform,
    Transform,
    Resize
)
from torch.nn.functional import interpolate
from skimage.transform import rescale
from pydicom import dcmread
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import torch.nn.functional as F
import h5py
from tqdm.auto import tqdm
from scipy import ndimage as ndi
import os

In [2]:
import pyrootutils
import hydra

root = pyrootutils.setup_root(search_from="./", indicator=[".git", "pyproject.toml"], pythonpath=True, dotenv=True)


In [3]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize("../configs/datamodule/", version_base="1.2")
data_module = hydra.compose("inference_bleed_cls.yaml")

In [4]:
datamodule = hydra.utils.instantiate(data_module)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Columns (1,27,83,121,140) have mixed types. Specify dtype option on import or set low_memory=False.


0it [00:00, ?it/s]

2023-08-11 14:11:30.350 | INFO     | src.datamodules.base_dataloader:setup:56 - Train set Length: 211
2023-08-11 14:11:30.351 | INFO     | src.datamodules.base_dataloader:setup:57 - Val set Length: 320
2023-08-11 14:11:30.352 | INFO     | src.datamodules.base_dataloader:setup:64 - Test set Length: 27397


In [5]:
val_ds = datamodule.val_ds

In [5]:
test_ds = datamodule.test_ds


In [6]:
from src.common.nn_modules.nets.slicewise.v3_slicelabels import V3Net , modify_model_state_dict

In [7]:
ckpt_path = "/data_nas5/qer/shubham/ich_checkpoints/ich_classification/runs/2023-08-01_04-40-47/checkpoints/epoch=9_step=260_val_loss=0.99.ckpt"


In [8]:
k = torch.load(ckpt_path,map_location = "cpu")
state_dict = k['state_dict']
new_state_dict = modify_model_state_dict(state_dict , "net.backbone" , "backbone")
new_state_dict = modify_model_state_dict(new_state_dict , "net.classification_head" , "classification_head")

In [9]:
model = V3Net()

Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.


In [10]:
model.load_state_dict(new_state_dict)
model.to("cuda:2")
model.eval()

V3Net(
  (backbone): Unet(
    (encoder): SENetEncoder(
      (layer0): Sequential(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (layer1): Sequential(
        (0): SEResNeXtBottleneck(
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, a

In [11]:
activation = torch.nn.Softmax(dim=1)

In [13]:
li =[]


In [ ]:
for i in tqdm(range(len(test_ds))) :
    data = test_ds[i]
#     if data['study_uid'] in test_study_uids :
#     if data['image'].shape[1] <= 75 : 
    image = data['image'].unsqueeze(0).to("cuda:2")
    sdh_outputs = model(image)
    sdh_model_score = activation(sdh_outputs['scan_label'])
    sdh_score = sdh_model_score.detach().cpu().numpy()
    del sdh_outputs
    del image
    torch.cuda.empty_cache()
    li.append({"StudyUID": data['study_uid'] ,"sdh_model_score_crop": sdh_score[0][1]})

        

  0%|          | 0/27397 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
len(li)

31251

In [18]:
pd.DataFrame.from_records(li).to_csv("v3net_crop_qure_25k.csv",index =False)

In [17]:
df_v3 = pd.DataFrame.from_records(li)

In [21]:
df_v3[df_v3.sdh_model_score_crop > 0.8]

,StudyUID,sdh_model_score_crop
8,1.2.840.113619.2.55.3.2831165733.883.136815380...,0.871872
65,ID_bd18514f0d,0.857413
101,ID_04028c6b7c,0.983080
274,1.2.840.113619.2.278.3.2831165733.263.13466766...,0.804354
301,1.2.840.114350.2.451.2.798268.2.2222575679147.1,0.991610
...,...,...
5076,1.2.840.113619.2.55.3.2831165733.97.1499138980.77,0.874287
5109,1.2.840.113619.2.55.3.2831165733.253.150285547...,0.811254
5149,1.2.840.113619.2.55.3.2831165735.699.147643222...,0.891603
5256,1.2.840.113619.2.278.3.2831165729.10.149188357...,0.930713


In [21]:
df_blend = pd.read_csv("output_blend_v3_only.csv")

In [23]:
df_blend_final = pd.concat([pd.read_csv("output_blend_v3_only.csv"), pd.read_csv("output_blend_v3_only_negative_ICH.csv") , pd.read_csv("output_blend_v3_only_qure25k.csv")])




In [25]:
df_blend_final.drop_duplicates(subset = ["StudyUID"]).to_csv("blend_hemorrhages_v3_qure25k_output",index =False)

In [26]:
df_blend_final

,StudyUID,blend_score,hemorrhages_v3
0,ID_baa6c286df,0.977944,0.996209
1,1.2.840.113619.2.404.3.4161469502.903.15782036...,0.981261,0.994918
2,ID_18b0309fe7,0.250083,0.117239
3,ID_43d8c37ac5,0.980400,0.989421
4,ID_d17802c522,0.989911,0.995339
...,...,...,...
31264,1.2.840.113619.2.55.3.2831165736.501.150136321...,0.892138,0.899195
31265,1.2.840.113619.2.55.3.2831165733.162.151056203...,0.071412,0.016011
31266,1.2.826.0.1.3680043.2.951.2403855,0.234757,0.220306
31267,1.3.39.19600601.2.2.999.20191111.184510715.431,0.200259,0.272953


In [28]:
df_blend[df_blend.hemorrhages_v3 <0.5]

,StudyUID,blend_score,hemorrhages_v3
2,ID_18b0309fe7,0.250083,0.117239
24,1.2.840.113711.574435.2.5388.593499525.26.2116...,0.363445,0.212433
39,1.2.840.113711.3345548.1.7496.603829109.26.211...,0.239377,0.113066
44,ID_3cd2c677b1,0.439568,0.203412
74,ID_1ab3043eab,0.149961,0.178798
...,...,...,...
5263,1.3.6.1.4.1.25403.52234588329.3540.20171023115...,0.401126,0.177737
5266,1.3.46.670589.33.1.63718693773919546700001.484...,0.278162,0.249045
5280,ID_e761b33102,0.309467,0.261955
5283,ID_42f6513ebb,0.218140,0.136541


In [30]:
len(set(df_v3[df_v3.sdh_model_score_crop < 0.5].StudyUID.values) - set(df_blend[df_blend.hemorrhages_v3 < 0.5].StudyUID.values))



93

In [32]:
len(set(df_blend[df_blend.hemorrhages_v3 < 0.5].StudyUID.values) -set(df_v3[df_v3.sdh_model_score_crop < 0.5].StudyUID.values))




37

In [19]:
df_fusion = pd.read_csv("fusion_v5.csv")

In [21]:
df_fusion[df_fusion.sdh_model_score < 0.5]

,StudyUID,sdh_model_score
10,ID_ec6cbbac41,0.064404
33,ID_73cb9b63f0,0.214579
47,1.2.840.113619.2.278.3.2831165733.718.13256219...,0.312155
53,1.2.840.113619.2.81.290.26072.18991.20171223.1...,0.132326
55,ID_8356684ce6,0.231986
...,...,...
5213,1.3.6.1.4.1.11157.7124.20191115165606.21390155...,0.041792
5214,1.2.840.113619.2.55.3.2831165803.2991.14994888...,0.140492
5272,1.2.840.113619.2.404.3.336209696.212.158288128...,0.161656
5280,ID_9adbf2f5f3,0.178090


In [ ]:
pd.read_csv("test_negative_ICH_sampled.csv")

In [7]:
os.path.exists("/data_nas3/processed/hct/sampling/ct_batches_all/batch_fts_2/sampled/1.3.12.2.1107.5.1.4.34169.4.0.7073610329675184.nii.gz")



False